![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [40]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, lit,to_date,concat_ws
from pyspark.sql.types import BooleanType
import numpy as np # Only if you plan to use np.NaN for numerical missing values
spark=SparkSession.builder.appName('datacamp').getOrCreate()
# Start coding here...

In [41]:
df = pd.read_csv("bank_marketing.csv")
df
# for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
#     print(col)
#     print("--------------")
#     print(df[col].value_counts())

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,nov,30,334,1,0,nonexistent,94.767,1.028,yes
41184,41184,46,blue-collar,married,professional.course,no,no,nov,6,383,1,0,nonexistent,94.767,1.028,no
41185,41185,56,retired,married,university.degree,no,yes,nov,24,189,2,0,nonexistent,94.767,1.028,no
41186,41186,44,technician,married,professional.course,no,no,nov,17,442,1,0,nonexistent,94.767,1.028,yes


In [42]:
df=spark.createDataFrame(df)

In [10]:
required_columns=['client_id','age','job','marital','education','credit_default',
                  'mortgage']
client_df=df.select(*required_columns)

client_df= client_df.withColumn("job", regexp_replace("job", r"\.", "_"))
client_df=client_df.withColumn('education',regexp_replace("education",r'\.','_'))
client_df=client_df.withColumn("education", when(col("education") == "unknown", lit(None)).otherwise(col("education")))
client_df=client_df.withColumn('credit_default',when(col('credit_default')=='yes',1).otherwise(0))
client_df=client_df.withColumn('mortgage',when(col('mortgage')=='yes',1).otherwise(0))
client_df.write.option('header',True).mode('overwrite').csv('client.csv')

25/05/29 19:24:47 WARN TaskSetManager: Stage 1 contains a task of very large size (1020 KiB). The maximum recommended task size is 1000 KiB.


In [51]:
required_columns=['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome']
campaign_df=df.select(*required_columns)
campaign_df=campaign_df.withColumn('previous_outcome',when(col('previous_outcome')=='yes',1).otherwise(0))
campaign_df=campaign_df.withColumn('campaign_outcome',when(col('campaign_outcome')=='yes',1).otherwise(0))
# df=df.withColumn('year',lit('2022'))
campaign_df=df.withColumn('last_contact_date',to_date(concat_ws("-", lit(2022), col("month"), col("day")), "yyyy-MMM-d"))
campaign_df.write.option('header',True).mode('overwrite').csv('campaign.csv')
# campaign_df.show(n=1,truncate=False)

25/05/29 19:48:27 WARN TaskSetManager: Stage 7 contains a task of very large size (1020 KiB). The maximum recommended task size is 1000 KiB.


In [52]:
required_columns=['client_id','cons_price_idx','euribor_three_months']
economics_df=df.select(*required_columns)
economics_df.write.option('header',True).mode('overwrite').csv('economics.csv')

25/05/29 19:50:02 WARN TaskSetManager: Stage 8 contains a task of very large size (1020 KiB). The maximum recommended task size is 1000 KiB.
